In [1]:
import pandas as pd

In [6]:
imr = pd.read_csv('../data/sage-IMR-records.csv', sep='\t')
imr.head()


,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,journal,volume,issue,issue_title,issue_page_range,issue_pub_date,issue_pub_year,publisher
0,Immigrants’ Attitudes toward their Children's ...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Peter D. Chimbos,"Chimbos, Peter D","Department of Sociology, Lakehead University, ...",5-17,"Mar 1, 1971",1971,Article,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
1,Immigrant Minorities in the Netherlands: Integ...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Christopher Bagley,"Bagley, Christopher","Institute of Psychiatry, London.",18-35,"Mar 1, 1971",1971,Article,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
2,Rural-Urban Migration in Latin America: A Fram...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Robert V. Kemper,"Kemper, Robert V","Department of Anthropology, University of Cali...",36-47,"Mar 1, 1971",1971,Article,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
3,Oriental Immigration: The Experience of the Ch...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Monica Boyd,"Boyd, Monica","Department of Sociology, Carleton University, ...",48-61,"Mar 1, 1971",1971,Article,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
4,Immigrant Absorption and Religion,https://journals.sagepub.com/doi/pdf/10.1177/0...,J. J. Mol,"Mol, J J",*,62-71,"Mar 1, 1971",1971,Documentation,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing


In [8]:
authorloos = imr.loc[imr.article_author.isnull()]

In [35]:
for wrd in ["[Aa]ustralia","[Nn]ew Zealand","USA", "United States","[Cc]anada",'[Nn]etherlands','\s[Mm]igr','[Ee]migr\w+', '[Ee]migration']:
    print (wrd, len(imr.loc[(imr.article_title.str.contains(f'{wrd}', regex=True)) & (imr.article_title.str.contains('Book Review'))]))

[Aa]ustralia 52
[Nn]ew Zealand 4
USA 2
United States 134
[Cc]anada 51
[Nn]etherlands 3
\s[Mm]igr 423
[Ee]migr\w+ 80
[Ee]migration 39


In [25]:
imr.loc[(imr.article_title.str.contains('[Ee]migra', regex=True)) & (imr.article_title.str.contains('Book Review'))]

,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,journal,volume,issue,issue_title,issue_page_range,issue_pub_date,issue_pub_year,publisher
12,Book Review: Slovenské vystahovalectvo (Slovak...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Joseph S. Roucek,"Roucek, Joseph S",City University of New York (Queensborough Com...,97-99,"Mar 1, 1971",1971,Book Reviews,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
16,Book Review: Emigrazione e comportamento politico,https://journals.sagepub.com/doi/pdf/10.1177/0...,Francesco P. Cerase,"Cerase, Francesco P",University of Rome,102-103,"Mar 1, 1971",1971,Book Reviews,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
17,Book Review: L'emigrazione italiana in Europa ...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Pellegrino Nazzaro,"Nazzaro, Pellegrino",Rochester Institute of Technology,103-104,"Mar 1, 1971",1971,Book Reviews,International Migration Review,5,1,"International Migration Review - Volume 5, Num...",NaN,"Mar 01, 1971",1971,Sage Publishing
75,Book Review: The Distant Magnet—European Emigr...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Joseph H. Bunzel,"Bunzel, Joseph H",New York State University College at Buffalo,453-453,"Dec 1, 1972",1972,Book Reviews,International Migration Review,6,4,"International Migration Review - Volume 6, Num...",NaN,"Dec 01, 1972",1972,Sage Publishing
96,Book Review: Emigración y Cambio Social,https://journals.sagepub.com/doi/pdf/10.1177/0...,Sergio Diaz Briquets,"Briquets, Sergio Diaz",Georgetown University,474-475,"Dec 1, 1972",1972,Book Reviews,International Migration Review,6,4,"International Migration Review - Volume 6, Num...",NaN,"Dec 01, 1972",1972,Sage Publishing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032,Book Review: Dear China: Emigrant Letters and ...,https://journals.sagepub.com/doi/full/10.1177/...,Kelvin E.Y. Low,"Low, Kelvin EY",National University of Singapore,611-612,"August 6, 2019",2019,Book Reviews,International Migration Review,54,2,"International Migration Review - Volume 54, Nu...",NaN,"Jun 01, 2020",2020,Sage Publishing
5084,Book Review: Leaving England: Essays on Britis...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Kerby A. Miller,"Miller, Kerby A",University of Missouri–Columbia,586-587,"Jun 1, 1995",1995,Book Reviews,International Migration Review,29,2,"International Migration Review - Volume 29, Nu...",NaN,"Jun 01, 1995",1995,Sage Publishing
5109,Book Review: Emigrants in Chains: A Social His...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Janet A. Nolan,"Nolan, Janet A","Loyola University, Chicago",652-652,"Sep 1, 1993",1993,Book Reviews,International Migration Review,27,3,"International Migration Review - Volume 27, Nu...",NaN,"Sep 01, 1993",1993,Sage Publishing
5110,Book Review: Pot Slovenskih Izseljencev na Tuj...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Joseph Velikonja,"Velikonja, Joseph",University of Washington,653-654,"Sep 1, 1993",1993,Book Reviews,International Migration Review,27,3,"International Migration Review - Volume 27, Nu...",NaN,"Sep 01, 1993",1993,Sage Publishing


In [27]:
(len(imr.loc[(imr.article_title.str.contains('Book Review'))])/len(imr))*100

44.72297910990009

In [11]:
list(authorloos.article_doi)

['https://journals.sagepub.com/doi/pdf/10.1177/019791837100500107',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791837100500128',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s02',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s03',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s04',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s05',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s06',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s07',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s08',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s09',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s10',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s11',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s12',
 'https://journals.sagepub.com/doi/pdf/10.1177/019791838001401s13',
 'https://journals.sagepub.com/doi/pdf/10.1177/0

In [12]:
import requests

In [ ]:
https://www-jstor-org.access.authkb.kb.nl/stable/

In [13]:
def doi2bib(doi):
  """
  Return a bibTeX string of metadata for a given DOI.
  """

  url = "http://dx.doi.org/" + doi

  headers = {"accept": "application/x-bibtex"}
  r = requests.get(url, headers = headers)

  return r.text

In [17]:
doi2bib('10.1177/019791838001401s15')

KeyboardInterrupt: 

In [ ]:
http://linker2.worldcat.org/?date=1971-03-01&wskey=yqJNtNXx2VoAseoNJXum1RIxJsSpIZCI1D2nFUFDXt8252OC3lIsWbJX91V2Sd1QqRhLXmhJtQSFfemW&issue=1&jKey=intemigrrevi&rft.institution_id=87606&linktype=best&spage=75&eissn=1747-7379&title=International+Migration+Review%2C+The&localstem=http%3A%2F%2Fauthkb.kb.nl%2Flogin%3FURL%3D&atitle=Review+of+Reviews&volume=5&linkScheme=jstor&jHome=https%3A%2F%2Fwww.jstor.org%2Fjournal%2Fintemigrrevi&url_ver=Z39.88-2004&issn=0197-9183&provider=jstor&epage=89&pkgName=asii&linkclass=to_article&doi=10.1177%2F019791837100500107